# DIGITAL RECOGNIZER

In this kaggle project, we have implemented neural networks to correctly identify digits from a dataset of tens of thousands of handwritten images. 

Data Set:

We downloaded the MNIST ("Modified National Institute of Standards and Technology) data from kaggle.com. Please refer the link below:
https://www.kaggle.com/c/digit-recognizer

Design Decisions:
1. We performed feature scaling using sklearn.
2. Cross validation was performed.
3. 2 hidden layers with 50 hidden nodes have been implemented.
4. Learning rate of 0.1 was set for initial iterations and later was reduced to 0.1.
5. Sigmoid function is used as activation function.

Import required libraries and set default parameters

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
%matplotlib inline
import random
no_of_features = 784
hidden_nodes = 50
output_nodes = 10
hidden_nodes2 = 50

Read train and test data into pandas data frames.

In [11]:
df = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

We have performed feature scaling below to avoid overflow and improved accuracy.

In [12]:
scaler = StandardScaler()
scaler.fit(df.drop('label',axis=1))
scaled_features = scaler.transform(df.drop('label',axis=1))
image_features = pd.DataFrame(scaled_features,columns=df.columns[1:])

In [13]:
train_features = image_features

In [14]:
train_label = df['label']

Cross-validation has been performed below.

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_features,train_label,test_size=0.30)

In [16]:
train_features = x_train
train_label = y_train
test_features = x_test
test_label = y_test
train_label = train_label.reset_index()
train_label = train_label.drop('index',axis=1)
train_features = train_features.reset_index()
train_features = train_features.drop('index',axis=1)

test_label = test_label.reset_index()
test_label = test_label.drop('index',axis=1)
test_features = test_features.reset_index()
test_features = test_features.drop('index',axis=1)
test_label = np.array(test_label)

Create matrix for storing weights for each layer.

In [17]:
def init_matrix(mat, row, column):
    mat =[]
    for i in range(row):
        arr =[]
        for j in range(column):
            val = random.uniform(-1.0,1.0)
            arr.append(val)
        mat.append(arr)
    return mat

We initialize the weight matrix with random weights between 1 and -1.

In [18]:
def initialize_weights():
    w1 = []
    w2 = []
    w3 = []
    w1 = init_matrix(w1,no_of_features,hidden_nodes)
    w2 = init_matrix(w2,hidden_nodes2,output_nodes)
    w3 = init_matrix(w3,hidden_nodes,hidden_nodes2)
    w1 = np.array(w1)
    w2 = np.array(w2)
    w3 = np.array(w3)
    return w1,w2,w3

Sigmoid function used for activation.

In [19]:
def sigmoid_function(z):
    z =  1.0/(1.0+np.exp(np.negative(z)))
    return z

Derivative of sigmoid function.

In [20]:
def diff_sigmoid_function(z):
    a = (z*(1-z))
    return a

Code for encoding the labels into format interpretable by neural network.

In [21]:
def actual_output(train_label):
    y = []
    for i in range(0,len(train_label)):
        
        if train_label[i]== 1:
            y.append((0,1,0,0,0,0,0,0,0,0))
        elif train_label[i] == 2:
            y.append((0,0,1,0,0,0,0,0,0,0))
        elif train_label[i]== 3:
            y.append((0,0,0,1,0,0,0,0,0,0))
        elif train_label[i] == 4:
            y.append((0,0,0,0,1,0,0,0,0,0))
        elif train_label[i]== 5:
            y.append((0,0,0,0,0,1,0,0,0,0))
        elif train_label[i] == 6:
            y.append((0,0,0,0,0,0,1,0,0,0))
        elif train_label[i]== 7:
            y.append((0,0,0,0,0,0,0,1,0,0))
        elif train_label[i]== 8:
            y.append((0,0,0,0,0,0,0,0,1,0))
        elif train_label[i] == 9:
            y.append((0,0,0,0,0,0,0,0,0,1))
        elif train_label[i] == 0:
            y.append((1,0,0,0,0,0,0,0,0,0))
    return y


Implementation of Feed forward to the neural network.

In [22]:
def feedforward(w1,w2,w3,x):
    z2 = np.dot(x,w1)
    a1 = sigmoid_function(z2)
    z4 = np.dot(a1,w3)
    a3 = sigmoid_function(z4)
    z3 = np.dot(a3,w2)
    a2 = sigmoid_function(z3)
    return a1,a2,a3,z4,z3,z2

Neural network construction that involves forward and backward propogation.
The network returns the trained weights.

In [40]:
def neural_nets(train_features,train_label):
    w1,w2,w3 = initialize_weights()
    a = np.array(train_label)
    y = actual_output(a)
    y = np.array(y)

    for j in range(0,30):
        print("Epoch no:",j)
        for i in range(0,len(train_label)):
                x = train_features.loc[i]
                ip = np.array(x[0:784])[np.newaxis]
                a1,a2,a3,z4,z3,z2 = feedforward(w1,w2,w3,ip)
                delta3 = np.multiply(((y[i]-a2)), diff_sigmoid_function(a2))
                djdw3 = np.dot(a3.T,delta3)
                delta2 = np.dot(delta3,w2.T) * diff_sigmoid_function(a3)
                djdw2 = np.dot(a1.T,delta2)
                delta1 = np.dot(delta2,w3.T) * diff_sigmoid_function(a1)
                djdw1 = np.dot(ip.T,delta1)
                learning_rate = 0.1 if j<20 else 0.01
                w1 = w1 + learning_rate * djdw1
                w2 = w2 + learning_rate * djdw3
                w3 = w3 + learning_rate * djdw2
    return w1,w2,w3

Below we have trained the neural network and stored the trained weights in w1,w2 and w3.
We would be using them in testing phase.

In [41]:
w1,w2,w3 = neural_nets(train_features,train_label)
print(w1)
print(w2)
print(w3)

Epoch no: 0
Epoch no: 1
Epoch no: 2
Epoch no: 3
Epoch no: 4
Epoch no: 5
Epoch no: 6
Epoch no: 7
Epoch no: 8
Epoch no: 9
Epoch no: 10
Epoch no: 11
Epoch no: 12
Epoch no: 13
Epoch no: 14
Epoch no: 15
Epoch no: 16
Epoch no: 17
Epoch no: 18
Epoch no: 19
Epoch no: 20
Epoch no: 21
Epoch no: 22
Epoch no: 23
Epoch no: 24
Epoch no: 25
Epoch no: 26
Epoch no: 27
Epoch no: 28
Epoch no: 29
[[-0.64735065  0.20819809  0.55473908 ..., -0.93511812  0.62841465
   0.6825555 ]
 [-0.84323945  0.88656638 -0.59937892 ..., -0.08587459  0.22735615
  -0.6321765 ]
 [ 0.10973446  0.76758299  0.22149643 ..., -0.32077813 -0.1362702
   0.9234647 ]
 ..., 
 [ 0.61013518 -0.97396975 -0.52837762 ...,  0.75256533  0.33498545
   0.43408105]
 [-0.11621277 -0.34956575 -0.89813486 ...,  0.91851742 -0.78713963
  -0.72898324]
 [ 0.35866426 -0.13678911  0.30080757 ...,  0.74261196 -0.93548971
   0.37253284]]
[[-1.78934992 -0.53864205  1.87749006  2.31930263 -0.68855665  2.36464021
  -3.14247716 -2.95250675 -2.58542823 -0.983085

Function module to perform testing using feed forward.

In [42]:
def test_nnet():

    predicted_array=[]
    label = [0,1,2,3,4,5,6,7,8,9]
    
   
    for i in range(0,len(test_label)):
        x = np.array(test_features.loc[i])
        z2 = np.dot(x,w1)
        a2 = sigmoid_function(z2)
        z4 = np.dot(a2,w3)
        a3 = sigmoid_function(z4)
        z3 = np.dot(a3,w2)
        yhat = sigmoid_function(z3)
        max = -99.99
        max_label = 0
        for j in range(0,len(yhat)):
            if yhat[j]>max:
                max = yhat[j]
                max_label = j
        if max_label==0:
             predicted_array.append(label[0])
        elif max_label==1:
             predicted_array.append(label[1])
        elif max_label==2:
             predicted_array.append(label[2])
        elif max_label==3:
             predicted_array.append(label[3])
        elif max_label==4:
             predicted_array.append(label[4])
        elif max_label==5:
             predicted_array.append(label[5])
        elif max_label==6:
             predicted_array.append(label[6])
        elif max_label==7:
             predicted_array.append(label[7]) 
        elif max_label==8:
             predicted_array.append(label[8])
        elif max_label==9:
             predicted_array.append(label[9]) 
    correct = 0.0
    for i in range(0,len(test_label)):
        if predicted_array[i] == test_label[i]:
             correct +=1
    print("-----"*18)
    print ("Predicted orientation of",int(correct),"images correctly out of",len(test_label),"test images.")
    print ("-----"*18)
    accuracy = float(float(correct)/len(test_label))
    print( "accuracy",accuracy*100,"%")

Function call to the testing module.

In [43]:
test_nnet()

------------------------------------------------------------------------------------------
Predicted orientation of 11625 images correctly out of 12600 test images.
------------------------------------------------------------------------------------------
accuracy 92.26190476190477 %


## CONCLUSION:

We observe an accuracy of 91-94% over the test set, which looks to be pretty good.
We can try using other activation functions like RELU and softmax in order to observe change in performance.
As a step for further exploration, we can implement the project using Convolutional neural networks and deep learning.